<a href="https://colab.research.google.com/github/hagijakobson/geometric-brownian-motion/blob/main/simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Preço de Ações: Modelagem com Processos Estocásticos e Simulações de Monte Carlo

## Instalações

In [1]:
#!pip install -q yfinance

## Bibliotecas

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import seaborn as sns
import yfinance as yf

## Simulações

In [3]:
def monte_carlo_previsao(ativo, dias_a_frente, simulacoes, legend=True):
    dataset = pd.DataFrame(yf.download(ativo, start='2023-06-01',end='2023-07-01')['Close'])

    dataset_normalizado = dataset.copy()
    for i in dataset:
        dataset_normalizado[i] = dataset[i] / dataset[i][0]

    dataset_taxa_retorno = np.log(1 + dataset_normalizado.pct_change())
    dataset_taxa_retorno.fillna(0, inplace=True)

    media = dataset_taxa_retorno.mean()
    variancia = dataset_taxa_retorno.var()

    drift = media - (0.5 * variancia)
    desvio_padrao = dataset_taxa_retorno.std()
    Z = stats.norm.ppf(np.random.rand(dias_a_frente, simulacoes))
    retornos_diarios = np.exp(drift.values + desvio_padrao.values * Z)

    previsoes = np.zeros_like(retornos_diarios)
    previsoes[0] = dataset.iloc[-1]

    for dia in range(1, dias_a_frente):
        previsoes[dia] = previsoes[dia - 1] * retornos_diarios[dia]

    figura = px.line(title = 'Previsões do Preço do Ativo - ' + ativo)
    for i in range(len(previsoes.T)):
        figura.add_scatter(y = previsoes.T[i], name = i, showlegend=legend)
    figura.show()

    return previsoes.T

In [4]:
asset = "BTC-USD"
days_ahead = 30
simulations = 1000

previsoes = monte_carlo_previsao(asset, days_ahead, simulations, False)

[*********************100%***********************]  1 of 1 completed


In [5]:
figura = px.line(title = 'Previsão do Valor Esperado do Preço do Ativo - ' + 'BTC-USD')
figura.add_scatter(y = previsoes.mean(axis=0), name = 'BTC-USD', showlegend=False)
figura.show()